In [1]:
from malt import MaltParser # source code from nltk library

In [175]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from tqdm import tqdm
import json
#from datasets import load_dataset
from gensim import utils
from collections import defaultdict

import re

from word import Word
# from malt.malt import MaltParser # source code from nltk library
from malt import MaltParser # source code from nltk library

def process_dataset(docs, stop_words, pre_words, malt_parser_version='../maltparser-1.7.2', model_version='engmalt.linear-1.7.mco'):
    # initalize malt parser model
    mp = MaltParser(malt_parser_version, model_version, tagger=nltk.pos_tag)

    #Add more preprocessing:
    # replace single smart quote with single straight quote, so as to catch stopword contractions
    docs = [re.sub("[\u2018\u2019]", "'", doc) for doc in docs] #replace qoute with regualar qoutations
    #it removes the digits
    # docs = [re.sub('\d+', '', doc) for doc in docs] 
    docs = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in docs]
    docs = [re.sub(r"http\S+", '', doc) for doc in docs]
    
    # create <doc_idx, tokenized_sent> list of sents
    sents = [
        (i, nltk.word_tokenize(sent))
        for i, doc in enumerate(docs)
        for sent in nltk.sent_tokenize(utils.to_unicode(str(doc).lower())) # convert doc to lowercase, and sentence tokenized.
    ]

    # unzip list of tuples
    doc_idxs, sents = zip(*sents)

    # create parser <generator> and loop through parser to produce dependency tree for each sentence
    parser = mp.parse_sents(sents, verbose=True)

    # define valid word
    stop_words.append('amp');stop_words.append('&amp');stop_words.append('&amp;')
    valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2

    # initalize dictionary for json output
    docs_dict = {
        'documents': dict((doc_idx, {'words': [], 'relns': [], 'arg2':[] , 'originaltext': []}) for doc_idx in doc_idxs),
        #'documents': dict((doc_idx, {'words': [], 'relns': []}) for doc_idx in doc_idxs),
        'vocab': [], 
        'vocab_relns': [],
        'vocab_arg':[],
    }
    # initalize vocab variables as sets (no duplicates)
    vocab = set()
    vocab_relns = set()
    vocab_arg = set()
    vocab_arg_test, vocab_relns_test = set(), set()

    i = 0
    # loop through list iterators
    for list_it in parser:
        tree = next(list_it)
        # check if valid tree, if not skip
        try:
            nodes = tree.nodes
        except:
            continue
        #print(nodes)
#         print('******000000*****')
        word_relns_hash = defaultdict(list)
        word_args_hash = defaultdict(list)
        for word_idx in nodes:
            if word_idx == 0: # skip first
                continue
#             if word_idx >= 13:
#                 pass
#             if nodes[word_idx]['word']=='persons.for':
#                 print('bad documents 1 : ', nodes[word_idx])
#                 return nodes[word_idx]
            deps = nodes[word_idx]['deps']
            
#             if (nodes[word_idx]['word'] in stop_words):
#                 continue
            # check for valid dependency relation
            if deps:
                for reln, idxs in deps.items():
                    for idx in idxs:
                        if reln =='prep':
                            #what if it has more than one deps? do function/stop words appear with more than one deps?@ZHILA I also need to check what if this prepositional term refers to several words what do do with that
#                             if valid_word(nodes[idx]['word']):
                            if nodes[idx]['word'] in prep_words:
                                reln = 'prep'+'.'+nodes[idx]['word']
                                try:
                                    if len(list(nodes[idx]['deps'].values()))==0:
                                        continue
                                    elif type(list(nodes[idx]['deps'].values())[0])==list:
                                        idx = list(nodes[idx]['deps'].values())[0][0]
                                    else:
                                        idx = list(nodes[idx]['deps'].values())[0]
                                except:
                                    print('Error')
                                    return (nodes, idx)
                            
                            else:
                                continue
                            
                        
                        elif (not valid_word(nodes[word_idx]['word'] or not valid_word(nodes[idx]['word']))):
                        #in stop_words or nodes[idx]['word'] in stop_words:
                            continue
#                         if nodes[word_idx]['word'] in stop_words or nodes[idx]['word'] in stop_words:
#                             continue
                            
                        dep_reln, gov_reln = f"{reln}.dep", f"{reln}.gov"
                        if gov_reln =="punct.gov":
                            continue
                        
                        
                        # add reln to word in hashmap
                        word_relns_hash[idx].append(dep_reln) # append to dep word
                        word_relns_hash[word_idx].append(gov_reln) # append to current word
                        word_args_hash[idx].append(nodes[word_idx]['word'])
                        word_args_hash[word_idx].append(nodes[idx]['word'])
                        #vocab_arg.add(nodes[idx]['word'])
                        #vocab_arg.add(nodes[word_idx]['word'])
                        #vocab_relns.add(dep_reln)
                        #vocab_relns.add(gov_reln)
                        
                        
        
        # check for valid hashmap
        if word_relns_hash:
            doc_idx = doc_idxs[i]

            # loop through hashmap items and append to dict for future storing
            for word_idx, relns in word_relns_hash.items():
                word = nodes[word_idx]['word']
                #relns = [reln for reln in relns if reln != "punct.gov"]
#                 if word =='persons.for':
#                     print(' some error in bad document: ', doc_idx)
#                     return doc_idx
                if valid_word(word):
                    vocab.add(word)
                    docs_dict['documents'][doc_idx]['words'].append(word)
                    docs_dict['documents'][doc_idx]['relns'].append(relns)
                    docs_dict['documents'][doc_idx]['originaltext'] = [docs[doc_idx]]
                    docs_dict['documents'][doc_idx]['arg2'].append(word_args_hash[word_idx])
                    try:
                        vocab_arg.update(word_args_hash[word_idx])
                        vocab_relns.update(relns)
                    except:
                        print(word_args_hash[word_idx])
#                         print(vocab_arg_test)
                        return 0
        
        i += 1

    docs_dict['vocab'] = list(vocab)
    docs_dict['vocab_relns'] = list(vocab_relns)
    docs_dict['vocab_arg'] = list(set(vocab_arg))
#     docs_dict['vocab_arg_test']=list(vocab_arg_test)
#     docs_dict['vocab_relns_test']= list(vocab_relns_test)

    return docs_dict

In [3]:
# set the stop words
import pickle
# with open("../Stopword_list",'rb') as read_file:
with open('extra_stopwords','rb') as read_file:
    more_stop_words = pickle.load(read_file)
# stop word initialization
# with open("data/utils/stopwords.txt") as f:
#     more_stop_words = f.read().splitlines()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(more_stop_words)

In [4]:
import mysql.connector

sql_db = mysql.connector.connect(host='127.0.0.1',user = 'covidAnalysis',password = 'k34p63MbDDcZ9yf4',
                                 database = 'covid19framing')
cursor = sql_db.cursor()

In [5]:
cursor.execute('show tables;')
results = cursor.fetchall()
# I changed the chosen column to be text_preproc and not text because of some issues in those rows wherein text_preproc is null and text exist...
cursor.execute("select * from articles where text_preproc is not NULL")
results = cursor.fetchall()
len(results)

3699

In [6]:
originaltext = [item[1].decode() for item in results] #originaltextoriginaltext[1]
corpus = pd.Series(originaltext)

In [125]:
sample_size = int(len(corpus) * .1)
sample_corpus = corpus.sample(n=sample_size)
len(sample_corpus)

369

In [229]:
import pickle
with open('prep_words.txt', "r") as file:
    prep_words = [line.strip() for line in file]
# print(prep_words)

In [176]:
docs_dict= process_dataset(sample_corpus, stop_words, prep_words)

-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Wed Aug 02 21:39:11 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    356MB
.          	     10	      4s	    367MB
.          	    100	      5s	    410MB
..........	   1000	     11s	    763MB
..........	   2000	     17s	    504MB
..........	   3000	     20s

In [173]:
valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2
valid_word('persons.for')

False

In [160]:
docs_dict= process_dataset(sample_corpus, stop_words, prep_words)

-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Wed Aug 02 21:23:29 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    365MB
.          	     10	      4s	    376MB
.          	    100	      5s	    421MB
..........	   1000	     12s	    781MB
..........	   2000	     17s	    538MB
..........	   3000	     20s

 some error in bad document:  45


In [177]:
json_object = json.dumps(tst, indent=4)
with open("test2_sample.json", "w") as f:
    f.write(json_object)

with open("test2_sample.json") as json_file:
    o = json.load(json_file)

In [166]:
docs_dict['documents']

TypeError: 'int' object is not subscriptable

In [232]:
# json_object = json.dumps(docs_dict, indent=4)
# with open("test_sample.json", "w") as f:
#     f.write(json_object)

with open("sample_10.json") as json_file:
    o = json.load(json_file)

In [234]:
len(o['documents'])

369

In [235]:
# see if all the relns are captured in vocabulary?
doc_objects = o['documents'] #words, reln, originaltext
docs = [ doc_objects[str(doc_id)]['words'] for doc_id in doc_objects ]
text = [ doc_objects[str(doc_id)]['originaltext'] for doc_id in doc_objects ]
doc_relns = [ doc_objects[str(doc_id)]['relns'] for doc_id in doc_objects ]
doc_arg2 = [ doc_objects[str(doc_id)]['arg2'] for doc_id in doc_objects ]
vocab = o['vocab']
vocab_relns = o['vocab_relns']
vocab_arg = o['vocab_arg']


In [236]:
output = []
def reemovNestings(l):
    for i in l:
        if type(i) == list:
            reemovNestings(i)
        else:
            output.append(i)
    return output
    
reemovNestings(doc_relns)

unique_reln = set(output)

len(unique_reln),len(vocab_relns)
#print relns that are not captured in doc_relns...
ls_reln_nan = [item for item in vocab_relns if item not in unique_reln]
ls_reln_zeros =  [item for item in unique_reln if item not in vocab_relns]
len(ls_reln_nan), len(ls_reln_zeros)
# 'prep.departamento.gov' in vocab_relns
ls_reln_nan, ls_reln_zeros

([], [])

In [237]:
output = []
reemovNestings(doc_arg2)
unique_arg2 = set(output)

len(unique_arg2),len(vocab_arg)
#print relns that are not captured in doc_relns...
ls_arg_nan = [item for item in vocab_arg if item not in unique_arg2]
ls_arg_zeros =  [item for item in unique_arg2 if item not in vocab_arg]
len(ls_arg_nan), len(ls_arg_zeros)
# 'prep.departamento.gov' in vocab_relns
# ls_arg_nan, ls_arg_zeros

(0, 0)

In [238]:
'persons.for' in vocab, 'prep.persons.for.dep' in unique_reln, 'persons' in vocab
# vocab[0]

(False, False, True)

In [228]:
# prep_relns

In [239]:
prep_relns = [item for item in unique_reln if item.startswith('prep')]
prep_relns
# bad_relns = ['prep.ahora.dep',
#  'prep.kansas.dep',
#  'prep.tratamiento.dep',
#  'prep.given.dep',
#  'prep.continues.dep',
#  'prep.ver.gov',
#  'prep.award.gov',
#  'prep.associationgovernor.dep',
#  'prep.volver.dep',
#  'prep.para.dep',
#  'prep.alfabetización.dep',
#  'prep.beginning.dep',
#  'prep.woodseast.dep',
#  'prep.backus.dep'
#             ]


['prep.until.dep',
 'prep.from.dep',
 'prep.as.gov',
 'prep.onto.gov',
 'prep.amid.dep',
 'prep.inside.gov',
 'prep.to.dep',
 'prep.following.gov',
 'prep.like.dep',
 'prep.near.gov',
 'prep.into.dep',
 'prep.outside.dep',
 'prep.into.gov',
 'prep.across.dep',
 'prep.around.dep',
 'prep.unlike.dep',
 'prep.before.dep',
 'prep.past.gov',
 'prep.despite.dep',
 'prep.upon.dep',
 'prep.around.gov',
 'prep.off.dep',
 'prep.over.gov',
 'prep.than.dep',
 'prep.against.dep',
 'prep.near.dep',
 'prep.through.gov',
 'prep.in.gov',
 'prep.toward.gov',
 'prep.via.gov',
 'prep.of.gov',
 'prep.from.gov',
 'prep.by.dep',
 'prep.without.gov',
 'prep.against.gov',
 'prep.since.gov',
 'prep.along.dep',
 'prep.to.gov',
 'prep.than.gov',
 'prep.within.dep',
 'prep.in.dep',
 'prep.via.dep',
 'prep.about.dep',
 'prep.for.gov',
 'prep.up.dep',
 'prep.below.gov',
 'prep.except.gov',
 'prep.with.gov',
 'prep.among.dep',
 'prep.with.dep',
 'prep.until.gov',
 'prep.within.gov',
 'prep.onto.dep',
 'prep.outside.g

In [226]:
# doc_relns[25]

In [224]:
doc_relns[25]
text[25]

["– Governor Ned Lamont today said that the State of Connecticut and its nonprofit partners are continuing to seek volunteers willing to help communities respond to the COVID-10 crisis. The state's volunteer recruitment effort is being expanded to bolster the ranks of volunteers needed at nonprofits that offer support to children and adults in programs for intellectual disabilities and behavioral health settings, as well as organizations that support senior citizens.To date, more than 5,000 medical volunteers and 1,600 non-medical volunteers have come forward to assist Connecticut's hospitals and health systems, and also provide support to nonprofits that provide essential human services. Volunteers have helped distribute food with food banks, provided support for those experiencing homelessness in shelters across the state, and delivered meals to homebound senior citizens in communities across Connecticut.Governor Lamont said that he is grateful for everyone who has stepped up to volu

In [217]:
# [item for item in doc_relns if 'prep.continues.dep' in  item]
# doc_relns[28:30]
doc_ids = []

c = 0
for items in doc_relns:
    for deps in items:
        if len([1 for d in deps if d in bad_relns])>0:
            r = [d for d in deps if d in bad_relns]
            print(c, r)
            doc_ids.append(c)
            break
    c += 1
# doc_ids

8 ['prep.kansas.dep']
13 ['prep.backus.dep']
25 ['prep.continues.dep']
65 ['prep.beginning.dep']
71 ['prep.beginning.dep']
72 ['prep.volver.dep']
77 ['prep.given.dep']
114 ['prep.beginning.dep']
147 ['prep.tratamiento.dep']
148 ['prep.beginning.dep']
231 ['prep.beginning.dep']
241 ['prep.ahora.dep']
246 ['prep.associationgovernor.dep']
272 ['prep.para.dep']
283 ['prep.woodseast.dep']
289 ['prep.ver.gov']
291 ['prep.ahora.dep']
319 ['prep.award.gov']
329 ['prep.beginning.dep']


In [222]:
# text[8]
text[25]

["– Governor Ned Lamont today said that the State of Connecticut and its nonprofit partners are continuing to seek volunteers willing to help communities respond to the COVID-10 crisis. The state's volunteer recruitment effort is being expanded to bolster the ranks of volunteers needed at nonprofits that offer support to children and adults in programs for intellectual disabilities and behavioral health settings, as well as organizations that support senior citizens.To date, more than 5,000 medical volunteers and 1,600 non-medical volunteers have come forward to assist Connecticut's hospitals and health systems, and also provide support to nonprofits that provide essential human services. Volunteers have helped distribute food with food banks, provided support for those experiencing homelessness in shelters across the state, and delivered meals to homebound senior citizens in communities across Connecticut.Governor Lamont said that he is grateful for everyone who has stepped up to volu

In [50]:
# dir(docs_dict)
docs_dict[1][86]

defaultdict(<function nltk.parse.dependencygraph.DependencyGraph.__init__.<locals>.<lambda>()>,
            {0: {'address': 0,
              'word': None,
              'lemma': None,
              'ctag': 'TOP',
              'tag': 'TOP',
              'feats': None,
              'head': None,
              'deps': defaultdict(list, {'null': [7]}),
              'rel': None},
             1: {'address': 1,
              'word': 'all',
              'lemma': '_',
              'ctag': 'DT',
              'tag': 'DT',
              'feats': '_',
              'head': 2,
              'deps': defaultdict(list, {}),
              'rel': 'det'},
             2: {'address': 2,
              'word': 'data',
              'lemma': '_',
              'ctag': 'NNS',
              'tag': 'NNS',
              'feats': '_',
              'head': 7,
              'deps': defaultdict(list, {'det': [1], 'prep': [3]}),
              'rel': 'nsubj'},
             7: {'address': 7,
              'word

In [22]:
# dir(docs_dict)
docs_dict
# we do not change these nodes.. but why some nodes are like "persons.for"??

defaultdict(<function nltk.parse.dependencygraph.DependencyGraph.__init__.<locals>.<lambda>()>,
            {0: {'address': 0,
              'word': None,
              'lemma': None,
              'ctag': 'TOP',
              'tag': 'TOP',
              'feats': None,
              'head': None,
              'deps': defaultdict(list, {'null': [7]}),
              'rel': None},
             1: {'address': 1,
              'word': 'all',
              'lemma': '_',
              'ctag': 'DT',
              'tag': 'DT',
              'feats': '_',
              'head': 2,
              'deps': defaultdict(list, {}),
              'rel': 'det'},
             2: {'address': 2,
              'word': 'data',
              'lemma': '_',
              'ctag': 'NNS',
              'tag': 'NNS',
              'feats': '_',
              'head': 7,
              'deps': defaultdict(list, {'det': [1], 'prep': [3]}),
              'rel': 'nsubj'},
             7: {'address': 7,
              'word

In [103]:
json_object = json.dumps(docs_dict, indent=4)
with open("sample_10.json", "w") as f:
    f.write(json_object)

In [91]:
# nodes, idx = docs_dict[0], docs_dict[1]
# # list(nodes[idx]['deps'].values()[0])
# # list(nodes[idx]['deps'].values())[0]
# type(list(nodes[2]['deps'].values())[0])
# (list(nodes[2]['deps'].values())[0][0])

1

In [71]:
# list(docs_dict[0][2]['deps'].values())[0][0]
# len(list(docs_dict[0][2]['deps'].values())[0]), len(list(docs_dict[0][14]['deps'].values())[0])

(1, 0)

In [98]:
len(docs_dict['vocab_arg']), len(docs_dict['vocab_relns']), len(docs_dict['vocab']), len(docs_dict)

(8449, 288, 6414, 6)

In [171]:
dir(docs_dict)
# list(docs_dict.values())[0]
# docs_dict[14]['deps']
# list(docs_dict[idx]['deps'].values())[0],  docs_dict[14]['word'], 
len(docs_dict['documents'][0]['words']), len(docs_dict['documents'][0]['relns']), len(docs_dict['documents'][0]['arg2'])
(docs_dict['documents'][0])

{'words': ['concord',
  'issued',
  'chris',
  'sununu',
  'order',
  'part',
  'governor',
  'emergency',
  'efforts',
  'state',
  'respond',
  'emergency',
  'order',
  'found',
  'information',
  'establishment',
  'along',
  'long',
  'additional',
  'term',
  'care',
  'stabilization',
  'program',
  'security',
  'new',
  'hampshire',
  'employment',
  'apply',
  'details',
  'providers',
  'qualifying',
  'guidance',
  'pursuant',
  'document',
  'order',
  'emergency'],
 'relns': [['tmod.dep'],
  ['tmod.gov',
   'tmod.gov',
   'nsubjpass.gov',
   'auxpass.gov',
   'dobj.gov',
   'prep.as.gov'],
  ['nsubjpass.dep', 'nn.gov'],
  ['auxpass.dep'],
  ['dobj.dep', 'nn.gov', 'num.gov'],
  ['prep.as.dep', 'prep.of.gov'],
  ['nn.dep'],
  ['nn.dep'],
  ['prep.of.dep', 'poss.gov', 'infmod.gov'],
  ['possessive.gov', 'poss.dep'],
  ['infmod.dep', 'prep.to.gov'],
  ['nn.dep'],
  ['nn.gov', 'num.gov', 'dep.gov'],
  ['dep.dep', 'dep.gov', 'advmod.gov'],
  ['prep.of.dep',
   'amod.gov',
   'a

In [57]:
# 'to' in docs_dict['vocab_arg']
# docs_dict['documents']
c = 0
for i in docs_dict['documents']:
    for j in range(len(docs_dict['documents'][i]['relns'])):
        if len(docs_dict['documents'][i]['relns'][j])!=len(docs_dict['documents'][i]['arg2'][j]):
            c+=1
print(c)

0


In [118]:
docs_dict['documents'][i]['relns'][j],  docs_dict['documents'][i]['arg2'][j]

(['conj.dep'], ['services'])

In [10]:
# docs_dict['documents'][1]['arg2']

In [49]:

len(docs_dict['documents'][2]['arg2']), len(docs_dict['documents'][2]['relns'])
# 'in' in docs_dict['documents'][0]['arg2'], 'to' in docs_dict['documents'][0]['arg2'], 'of' in docs_dict['documents'][0]['arg2']
docs_dict['documents'][2]

{'words': ['governor',
  'announced',
  'plan',
  'today',
  'include',
  'lamont',
  'comprehensive',
  'put',
  'work',
  'million',
  'federal',
  'state',
  'resources',
  'providing',
  'assistance',
  'emergency',
  'renters',
  'homeowners',
  'residential',
  'landlords',
  'impacted',
  'program',
  'public',
  'health',
  'million',
  'rental',
  'assistance',
  'administered',
  'include',
  'connecticut',
  'residents',
  'impacted',
  'housing',
  'provide',
  'payments',
  'approved',
  'applicants',
  'tenant',
  'households',
  'denied',
  'insurance',
  'unemployment',
  'million',
  'help',
  'insured',
  'act',
  'extending',
  'bringing',
  'eviction',
  'prevention',
  'renters',
  'emergency',
  'public',
  'health',
  'million',
  'provide',
  'federally',
  'administered',
  'program',
  'assistance',
  'administered',
  'relief',
  'mortgage',
  'suffered',
  'homeowners',
  'impacts',
  'whose',
  'mortgages',
  'chfa',
  'connecticut',
  'authority',
  'housi

In [40]:
len(docs_dict['documents'][0]['words']), len(docs_dict['documents'][0]['relns']), len(docs_dict['documents'][0]['arg2']), len(docs_dict['documents'])

(151, 151, 151, 739)

In [38]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from tqdm import tqdm
import json
#from datasets import load_dataset
from gensim import utils
from collections import defaultdict

import re

from word import Word
# from malt.malt import MaltParser # source code from nltk library
from malt import MaltParser # source code from nltk library

def process_dataset_test(docs, stop_words, malt_parser_version='../maltparser-1.7.2', model_version='engmalt.linear-1.7.mco'):
    # initalize malt parser model
    mp = MaltParser(malt_parser_version, model_version, tagger=nltk.pos_tag)
    
    #Add more preprocessing:
    # replace single smart quote with single straight quote, so as to catch stopword contractions
    docs = [re.sub("[\u2018\u2019]", "'", doc) for doc in docs] #replace qoute with regualar qoutations
    #it removes the digits
    # docs = [re.sub('\d+', '', doc) for doc in docs] 
    docs = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in docs]
    docs = [re.sub(r"http\S+", '', doc) for doc in docs]
    
    # create <doc_idx, tokenized_sent> list of sents
    sents = [
        (i, nltk.word_tokenize(sent))
        for i, doc in enumerate(docs)
        for sent in nltk.sent_tokenize(utils.to_unicode(str(doc).lower())) # convert doc to lowercase, and sentence tokenized.
    ]

    # unzip list of tuples
    doc_idxs, sents = zip(*sents)

    # create parser <generator> and loop through parser to produce dependency tree for each sentence
    parser = mp.parse_sents(sents, verbose=True)

    # define valid word
    stop_words.append('amp');stop_words.append('&amp');stop_words.append('&amp;')
    valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2

    # initalize dictionary for json output
    docs_dict = {
        'documents': dict((doc_idx, {'words': [], 'relns': [], 'arg2':[] , 'originaltext': []}) for doc_idx in doc_idxs),
        #'documents': dict((doc_idx, {'words': [], 'relns': []}) for doc_idx in doc_idxs),
        'vocab': [], 
        'vocab_relns': [],
        'vocab_arg':[],
    }
    # initalize vocab variables as sets (no duplicates)
    vocab = set()
    vocab_relns = set()
    vocab_arg = set()
    vocab_arg_test, vocab_relns_test = set(), set()

    i = 0
    # loop through list iterators
    for list_it in parser:
        tree = next(list_it)
        # check if valid tree, if not skip
        try:
            nodes = tree.nodes
        except:
            continue
        print(nodes)
        #node has these elements:
        # address, word, lemma, ctag, tag, feats, head, deps, reln
        print('**** **** **** ******** **** **** ****')
        word_relns_hash = defaultdict(list)
        word_args_hash = defaultdict(list)
        for word_idx in nodes:
            if word_idx == 0: # skip first
                continue
#             print(nodes[word_idx])
            deps = nodes[word_idx]['deps']
#             print('*******')
#             print(deps)
#             exit()

            # check for valid dependency relations
            if deps:
                for reln, idxs in deps.items():
                    for idx in idxs:
                        dep_reln, gov_reln = f"{reln}.dep", f"{reln}.gov"
                        
                        #REMMEMBER TO REMOVE THIS IF WE DO NOT WANT TO GO THIS ROUTE.
                        #here, we need to check if reln is punct, or if arg2 is puntuation and continue and not add them in these cases..
                        if gov_reln =="punct.gov" or nodes[idx]['word'] in stop_words or nodes[word_idx]['word'] in stop_words:
                            continue
                            
                        # add relations to vocab
                        vocab_relns.add(dep_reln)
                        vocab_relns.add(gov_reln)
                        
                        # add reln to word in hashmap
                        word_relns_hash[idx].append(dep_reln) # append to dep word
                        word_relns_hash[word_idx].append(gov_reln) # append to current word
                        word_args_hash[idx].append(nodes[word_idx]['word'])
                        word_args_hash[word_idx].append(nodes[idx]['word'])
                        vocab_arg.add(nodes[idx]['word'])
                        vocab_arg.add(nodes[word_idx]['word'])
                        #vocab_relns.add(dep_reln)
                        #vocab_relns.add(gov_reln)
                        
                        
        
        # check for valid hashmap
        if word_relns_hash:
            doc_idx = doc_idxs[i]

            # loop through hashmap items and append to dict for future storing
            for word_idx, relns in word_relns_hash.items():
                word = nodes[word_idx]['word']
                #relns = [reln for reln in relns if reln != "punct.gov"]
                if valid_word(word):
                    vocab.add(word)
                    docs_dict['documents'][doc_idx]['words'].append(word)
                    docs_dict['documents'][doc_idx]['relns'].append(relns)
                    docs_dict['documents'][doc_idx]['originaltext'] = [docs[doc_idx]]
                    docs_dict['documents'][doc_idx]['arg2'].append(word_args_hash[word_idx])
                    try:
                        vocab_arg_test.update(word_args_hash[word_idx])
                        vocab_relns_test.update(relns)
                    except:
                        print(word_args_hash[word_idx])
                        print(vocab_arg_test)
                        return 0
        
        i += 1

    
    docs_dict['vocab'] = list(vocab)
    docs_dict['vocab_relns'] = list(vocab_relns)
    docs_dict['vocab_arg'] = list(set(vocab_arg))
    docs_dict['vocab_arg_test']=list(vocab_arg_test)
    docs_dict['vocab_relns_test']= list(vocab_relns_test)

    return docs_dict

In [39]:
docs_dict = process_dataset_test(sample_corpus[0:1], stop_words)

-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Mon Jul 31 11:28:25 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    365MB
.          	     10	      4s	    365MB
           	     16	      4s	    376MB
Parsing time: 00:00:02 (2905 ms)
Finished: Mon Jul 31 11:28:29 EDT 2023


defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7f107f693e50>, {0: {'address': 0, 'word': None, 'lemma': None, 'ctag': 'TOP', 'tag': 'TOP', 'feats': None, 'head': None, 'deps': defaultdict(<class 'list'>, {'null': [3]}), 'rel': None}, 1: {'address': 1, 'word': 'michigan', 'lemma': '_', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 2, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'nn'}, 2: {'address': 2, 'word': 'companies', 'lemma': '_', 'ctag': 'NNS', 'tag': 'NNS', 'feats': '_', 'head': 3, 'deps': defaultdict(<class 'list'>, {'nn': [1]}), 'rel': 'nsubj'}, 3: {'address': 3, 'word': 'need', 'lemma': '_', 'ctag': 'VBP', 'tag': 'VBP', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'nsubj': [2], 'xcomp': [5], 'punct': [19, 31], 'prep': [20]}), 'rel': 'null'}, 4: {'address': 4, 'word': 'to', 'lemma': '_', 'ctag': 'TO', 'tag': 'TO', 'feats': '_', 'head': 5, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'aux'}, 5: {'address': 5, 'word': 'fill', 'lem

In [46]:
print(sample_corpus[0:1].to_string())

2318    Michigan companies need to fill thousands of c...
